# Web Scraping with Beautiful Soup

This tutorial will provide a very basic introduction to using the Beautiful Soup package to scrape text data from the web. 

## Installation

In a terminal, install Beautiful Soup if necessary by running <pre><code>conda install beautiful-soup</code></pre>

## Retrieving HTML

The general idea behind web scraping is to retrieve data that exists on a website, and convert it into a format that is usable for analysis. Webpages are rendered by the brower from HTML and CSS code, but much of the information included in the HTML underlying any website is not interesting to us.

We begin by reading in the source code for a given web page and creating a Beautiful Soup object with the BeautifulSoup function.

### urllib2
urllib2 is a module for working with urls, we will use it to open connections to urls and retrieve the webpage source.

In [5]:
from bs4 import BeautifulSoup
import urllib2
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false').read()
soup = BeautifulSoup(page, "html.parser")
print type(soup)

<class 'bs4.BeautifulSoup'>


The soup object contains all of the HTML in the original document.

In [6]:
print soup.prettify()[0:1000]

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml">
 <head>
  <script src="http://sports-ak.espn.go.com/sports/optimizely.js">
  </script>
  <meta charset="utf-8">
   <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible">
    <link href="http://a.espncdn.com/prod/assets/icons/E.svg" mask="" rel="icon" sizes="any">
     <meta content="#CC0000" name="theme-color">
      <script type="text/javascript">
       if(true && navigator && navigator.userAgent.toLowerCase().indexOf("teamstream") >= 0) {
        window.location = 'http://a.m.espn.go.com/mobilecache/general/apps/sc';
    }
      </script>
      <title>
       2015-16 Regular Season NBA Player Stats and League Leaders - Scoring Per Game - National Basketball Association - ESPN
      </title>
      <meta content="xuj1ODRluWa0frM-BjIr_aSHoUC7HB5C1MgmYAM_GkA" name="google-site-verification"/>
      <meta content="B1FEB7C682C46C8FCDA3130F3D18AC28" name="msvalidate.01"/>
      <meta content="noodp" name="googlebot

## Parsing HTML

By "parsing HTML", we mean pulling out only the relevant tags/attributes for our analysis.  What Beautiful Soup does is provide a handy bunch of methods for doing this efficiently.

### find method

The find method will search for and return the first tag matching your corresponding search criteria, if it exists.  You can specify tag and attribute info etc.  There is also a findAll method that will return a collection of tags matching your query.

In [7]:
table_div = soup.find(id='my-players-table')
print table_div.prettify()[0:4000]

<div class="col-main" id="my-players-table">
 <div class="mod-container mod-table">
  <div class="mod-header stathead">
   <h4>
    Points Per Game Leaders - All Players
   </h4>
  </div>
  <div class="mod-content">
   <table cellpadding="3" cellspacing="1" class="tablehead">
    <tr align="right" class="colhead">
     <td align="left" style="width:20px;">
      RK
     </td>
     <td align="left">
      PLAYER
     </td>
     <td align="left">
      TEAM
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
       GP
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
       MPG
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
       PTS

Notice that the object returned by the find method is just another inner HTML structure, which we can step through just like we would have with the original soup object.  We've gone to the location in the webpage where the table that we seek starts, now we can use find again to get to the table data.

In [10]:
table_div = soup.find(id='my-players-table')
table = table_div.find("table")
print table

<table cellpadding="3" cellspacing="1" class="tablehead">
<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/play

### Searching by Attributes

Now that we have the table object, we need to step through the rows.  First we'll find the header row so we can populate what the field names will be in our data.  Here we're searching for tags under the table tag whose class attritbute is "colhead".  

In [11]:
table_head = table.find(attrs={"class":'colhead'})
print table_head.prettify()

<tr align="right" class="colhead">
 <td align="left" style="width:20px;">
  RK
 </td>
 <td align="left">
  PLAYER
 </td>
 <td align="left">
  TEAM
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
   GP
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
   MPG
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
   PTS
  </a>
 </td>
 <td>
  <span title="Field Goals Made-Attempted Per Game">
   FGM-FGA
  </span>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">
   FG%
  </a>
 </td>
 <td>
  <span title="3-Point Field Goals Made-Attempted Per Game">
   3PM-3PA
  </span>
 </td>
 <t

Now we find the actual values by searching for the 'td' tags, which is the tag for table data.

In [12]:
table_head = table.find(attrs={"class":'colhead'})
header_cols = table_head.findAll('td')
print header_cols

[<td align="left" style="width:20px;">RK</td>, <td align="left">PLAYER</td>, <td align="left">TEAM</td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td>, <td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td>, <td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qualified/false" t

Finally, we step through these columns and save them to a list to be used later.  We'll ignore the rank column (RK) because that doesn't give us anything we want later.  We also separate the **PLAYER** column into **PLAYER** and **POSITION**.

In [ ]:
cols = []
for header_col in header_cols:
    val = header_col.string
    if val != 'RK':
        cols.append(val)
    if val == 'PLAYER':
        cols.append('POSITION')
print cols

### Stepping Through a Table

The table rows are indicated by the tag 'tr'.  Again we can find them all and iterate through them.  Within each row we iterate through the respective columns.

In [14]:
table_rows = table.findAll('tr')
print table_rows

[<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qu

We will save our results in 2 different ways to demonstrate how we can handle both.  The first will be a list of dicts where the key is the field name and the value is the field value.  The second will just be a list of lists of stats with no field name values (we've already defined them earlier).

In [15]:
players_stats_dicts = []
players_stats_array = []
for row in table_rows:
    if row.attrs['class'][0]=='colhead':
        continue
    player_stats = []
    row_cols = row.find_all('td')
    col_vals = []
    player_col = row_cols[1]
    player_name = player_col.find('a').string
    player_position = player_col.contents[1]
    player_position = player_position.split(' ')[1]
    player_stats.append(player_name)
    player_stats.append(player_position)
    for i in range(2, len(row_cols)):
        stat = row_cols[i].string
        player_stats.append(stat)
    players_stats_array.append(player_stats)
    player_stats = dict(zip(cols, player_stats))
    players_stats_dicts.append(player_stats)
print players_stats_dicts[0:5]        

NameError: name 'cols' is not defined

Here we've used the zip function to combine pairs of lists into tuples, and then transformed that into a dict to get a dictionary of FIELD --> VALUE for every player in the table.

Beautiful Soup has many other features, including the ability to step up, down, and sideways in the HTML tree and basically search for any tags, attributes, or values.  For more, take a look at the [Documentation](http://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## Load into Pandas

Let's load our scraped data into Pandas and take a look at it.  Here is the first way we can do it, simply directly from the dictionary we defined.

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(players_stats_dicts)
df.head()

Now here is a 2nd way we can do it.  We convert the 2D stats array into a numpy array and create a Pandas dataframe from it along with the list of column headers we defined earlier.

In [ ]:
np_array = np.array(players_stats_array)
df = pd.DataFrame(np_array, columns=cols)
df.head()

## Exercise

The goal of this exercise is to combine the scoring and assists statistics for every player in the NBA in 2014-2015.  The end result will have them in a pandas dataframe with the fields from both pages for every player.

The general steps should be as follows:
- Create a function get_cols that retrieves the names of the header columns given a table element (skip the ranks, split the positions)
- Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  You can use either the dict approach or the numpy array approach.
- Write a python loop to loop through the various pages and call these functions on the appropriate urls so that you can retrieve every player (rather than just the top few).
- Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them
- Use the pandas.DataFrame.merge() function to join your 2 pandas dataframes together and get a total result

In [39]:
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
import numpy as np

pagenum_index = 41
pagenum = 0
main_data = []
data = []
scoring_url = 'http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/'
assist_url = 'http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/'
urls = [scoring_url, assist_url]

def loop_over_data_url(url):
    pagenum = 0
    
    while True:
        page = urllib2.urlopen(url + str(pagenum)).read()
        soup = BeautifulSoup(page, "html.parser")

        table_div = soup.find(id='my-players-table')
        table = table_div.find("table")

        data.append(get_data(table))

        pagenum = (pagenum + pagenum_index)

        if pagenum == 401:
            break
            
    return data

def get_cols(table):
    table_head = table.find(attrs={"class":'colhead'})
    header_cols = table_head.findAll('td')
    
    cols = []
    for header_col in header_cols:
        val = header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')
    return cols

def get_data(table):
    
    cols = get_cols(table)
    
    table_rows = table.findAll('tr')
    
    players_stats_dicts = []
    players_stats_array = []
    for row in table_rows:
        if row.attrs['class'][0] == 'colhead':
            continue
        player_stats = []
        row_cols = row.find_all('td')
        col_vals = []
        player_col = row_cols[1]
        player_name = player_col.find('a').string
        player_position = player_col.contents[1]
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        players_stats_array.append(player_stats)
        player_stats = dict(zip(cols, player_stats))
        players_stats_dicts.append(player_stats)
    return players_stats_dicts


for url in urls:
    main_data.append(loop_over_data_url(url))

print "Scoring df:"
df = pd.DataFrame.from_dict(main_data[0])
df.head()

print "Assist df:"
df = pd.DataFrame.from_dict(main_data[1])
df.head()


AttributeError: 'NoneType' object has no attribute 'findAll'